In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, DateTime, Date, Time
from sqlalchemy.orm import Session

In [3]:
import datetime as dt

In [4]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [11]:
# Creates Classes which will serve as the anchor points for our Table
class SP_Data_V2(Base):
    __tablename__ = 'sp_data_v2'
    id = Column(Integer, primary_key=True)
    date = Column(Date)
    open = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    adj_close = Column(Float)
    volume = Column(Integer)

C:\Users\lande\Anaconda3\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.SP_Data_V2, and will be replaced in the string-lookup table.
  item.__name__


In [12]:
# Create Database Connection
engine = create_engine("sqlite:///../../db/bulls_bears.sqlite")
conn = engine.connect()

In [13]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [7]:
# Use this to clear out the db
# ----------------------------------
Base.metadata.drop_all(engine)
Base.metadata.clear()

In [14]:
from sqlalchemy import inspect

In [15]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [16]:
# Collect the names of tables within the database
inspector.get_table_names()

['sp_data', 'sp_data_v2']

In [17]:
# Using the inspector to print the column names within the 'tweet_data' table and its types
columns = inspector.get_columns('sp_data_v2')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
date DATE
open FLOAT
high FLOAT
low FLOAT
close FLOAT
adj_close FLOAT
volume INTEGER


In [18]:
data_file = "../../datasets/SAP500_updated.csv"

In [19]:
sp_df = pd.read_csv(data_file)

In [20]:
sp_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-03,2251.570068,2263.879883,2245.129883,2257.830078,2257.830078,3770530000
1,2017-01-04,2261.600098,2272.820068,2261.600098,2270.750000,2270.750000,3764890000
2,2017-01-05,2268.179932,2271.500000,2260.449951,2269.000000,2269.000000,3761820000
3,2017-01-06,2271.139893,2282.100098,2264.060059,2276.979980,2276.979980,3339890000
4,2017-01-09,2273.590088,2275.489990,2268.899902,2268.899902,2268.899902,3217610000


In [21]:
sp_df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [22]:
sp_df["Date"] = sp_df["Date"].apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%d").date())

In [23]:
sp_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-03,2251.570068,2263.879883,2245.129883,2257.830078,2257.830078,3770530000
1,2017-01-04,2261.600098,2272.820068,2261.600098,2270.750000,2270.750000,3764890000
2,2017-01-05,2268.179932,2271.500000,2260.449951,2269.000000,2269.000000,3761820000
3,2017-01-06,2271.139893,2282.100098,2264.060059,2276.979980,2276.979980,3339890000
4,2017-01-09,2273.590088,2275.489990,2268.899902,2268.899902,2268.899902,3217610000


In [24]:
sp_df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [25]:
sp_df = sp_df.rename(columns={"Date": "date", "Open": "open", "High": "high",
                              "Low": "low", "Close": "close",
                              "Adj Close": "adj_close", "Volume": "volume"})

In [26]:
#Export data to sqlite
sp_df.to_sql('sp_data_v2', con=engine, if_exists = 'append', index = True, index_label = 'id')

In [27]:
engine.execute("SELECT * FROM sp_data_v2 LIMIT 15").fetchall()

[(0, '2017-01-03', 2251.570068, 2263.8798829999996, 2245.1298829999996, 2257.830078, 2257.830078, 3770530000),
 (1, '2017-01-04', 2261.600098, 2272.820068, 2261.600098, 2270.75, 2270.75, 3764890000),
 (2, '2017-01-05', 2268.179932, 2271.5, 2260.449951, 2269.0, 2269.0, 3761820000),
 (3, '2017-01-06', 2271.139893, 2282.100098, 2264.060059, 2276.97998, 2276.97998, 3339890000),
 (4, '2017-01-09', 2273.590088, 2275.48999, 2268.899902, 2268.899902, 2268.899902, 3217610000),
 (5, '2017-01-10', 2269.719971, 2279.27002, 2265.27002, 2268.899902, 2268.899902, 3638790000),
 (6, '2017-01-11', 2268.600098, 2275.320068, 2260.830078, 2275.320068, 2275.320068, 3620410000),
 (7, '2017-01-12', 2271.139893, 2271.780029, 2254.25, 2270.439941, 2270.439941, 3462130000),
 (8, '2017-01-13', 2272.73999, 2278.679932, 2271.51001, 2274.639893, 2274.639893, 3081270000),
 (9, '2017-01-17', 2269.139893, 2272.080078, 2262.810059, 2267.889893, 2267.889893, 3584990000),
 (10, '2017-01-18', 2269.139893, 2272.01001, 2263.

In [28]:
pd.read_sql("SELECT * FROM sp_data_v2 ORDER BY date DESC LIMIT 15", con = conn).reset_index(drop = True)

,id,date,open,high,low,close,adj_close,volume
0,900,2020-07-31,3270.449951,3272.169922,3220.260010,3271.120117,3271.120117,5117260000
1,899,2020-07-30,3231.760010,3250.919922,3204.129883,3246.219971,3246.219971,4254010000
2,898,2020-07-29,3227.219971,3264.739990,3227.219971,3258.439941,3258.439941,4676300000
3,897,2020-07-28,3234.270020,3243.719971,3216.169922,3218.439941,3218.439941,4027890000
4,896,2020-07-27,3219.840088,3241.429932,3214.250000,3239.409912,3239.409912,3963910000
5,895,2020-07-24,3218.580078,3227.260010,3200.050049,3215.629883,3215.629883,3894900000
6,894,2020-07-23,3271.639893,3279.989990,3222.659912,3235.659912,3235.659912,4290460000
7,893,2020-07-22,3254.860107,3279.320068,3253.100098,3276.020020,3276.020020,4255190000
8,892,2020-07-21,3268.520020,3277.290039,3247.770020,3257.300049,3257.300049,4547960000
9,891,2020-07-20,3224.290039,3258.610107,3215.159912,3251.840088,3251.840088,3971200000


In [29]:
session = Session(engine)

In [30]:
session.query(func.max(SP_Data_V2.date)).all()

[(datetime.date(2020, 7, 31))]

In [31]:
session.query(func.min(SP_Data_V2.date)).all()

[(datetime.date(2017, 1, 3))]